# Evaluación de Eficiencia de Modelos

En este cuaderno se evalúa la eficiencia computacional de los modelos entrenados: **Random Forest**, **Regresión Logística** y **SVM**. Esta evaluación se centra en:

- Tiempo de inferencia (tiempo que tarda cada modelo en realizar una predicción)
- Capacidad de uso en tiempo real o entornos con recursos limitados
- Especificaciones del sistema utilizado para la evaluación

Este análisis complementa la evaluación métrica tradicional (F1-score, AUC) proporcionando una visión más práctica sobre el comportamiento de los modelos en producción.


## Carga de Modelos Entrenados

En este notebook se comparan los tres modelos entrenados previamente: **Random Forest**, **Regresión Logística** y **SVM (kernel RBF con subconjunto reducido)**.

Se cargan directamente desde los archivos `.pkl` guardados durante el entrenamiento, permitiendo reutilizarlos sin volver a entrenarlos. Esto garantiza reproducibilidad y eficiencia en la evaluación final.


In [2]:
import joblib

# Cargar modelos previamente guardados
rf_model = joblib.load("../models/random_forest_model.pkl")
lr_model = joblib.load("../models/logistic_regression_model.pkl")
svm_model = joblib.load("../models/svm_model.pkl")

print("Modelos cargados correctamente:")
print("- Random Forest")
print("- Logistic Regression")
print("- SVM (entrenado con 10,000 muestras)")


Modelos cargados correctamente:
- Random Forest
- Logistic Regression
- SVM (entrenado con 10,000 muestras)


In [4]:
from sklearn.metrics import f1_score, roc_auc_score

# Asegúrate de que tienes X_test e y_test cargados desde el dataset limpio
import pandas as pd

# Cargar el dataset limpio y dividir
df = pd.read_csv("../archive/cleaned_dataset.csv")
X = df.drop(columns=["Label"])
y = df["Label"]

# División en test para evaluación (20% del total, como antes)
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Obtener predicciones y probabilidades (cuando sea posible)
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

y_pred_lr = lr_model.predict(X_test)
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]

y_pred_svm = svm_model.predict(X_test)
y_score_svm = svm_model.decision_function(X_test)  # no .predict_proba()

# Calcular métricas
resultados = pd.DataFrame({
    "Modelo": ["Random Forest", "Logistic Regression", "SVM (RBF, 10k)"],
    "F1-score": [
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_svm)
    ],
    "AUC": [
        roc_auc_score(y_test, y_prob_rf),
        roc_auc_score(y_test, y_prob_lr),
        roc_auc_score(y_test, y_score_svm)
    ]
})

# Redondear para mejor visualización
resultados["F1-score"] = resultados["F1-score"].round(4)
resultados["AUC"] = resultados["AUC"].round(4)

resultados


,Modelo,F1-score,AUC
0,Random Forest,0.9999,1.0000
1,Logistic Regression,0.9916,0.9992
2,"SVM (RBF, 10k)",0.8252,0.6713


## Análisis Comparativo de Métricas de Clasificación

La siguiente tabla muestra los resultados de evaluación obtenidos por los tres modelos entrenados y probados sobre el mismo conjunto de datos:

| Modelo              | F1-score | AUC   |
|---------------------|----------|-------|
| Random Forest       | 0.9999   | 1.0000 |
| Logistic Regression | 0.9916   | 0.9992 |
| SVM (RBF, 10k)      | 0.8252   | 0.6713 |

### Interpretación:

- **Random Forest** obtuvo resultados perfectos en ambas métricas, lo que indica una separación total entre clases. Esto se alinea con los resultados observados en la matriz de confusión y curva ROC, donde no se produjeron errores. Sin embargo, su interpretabilidad es menor que la de otros modelos.

- **Regresión Logística** también mostró un rendimiento prácticamente perfecto. Aunque es un modelo mucho más simple, logró un F1-score y AUC cercanos a 1.0. Este resultado refuerza la idea de que el dataset CICIDS2017 permite separar las clases incluso con modelos lineales bien ajustados. Además, su interpretabilidad y eficiencia computacional son ventajas destacadas.

- **SVM** (entrenado con un subconjunto de 10,000 muestras debido a restricciones de tiempo y coste computacional) obtuvo un rendimiento claramente inferior. El F1-score de 0.83 y AUC de 0.67 indican que, aunque el modelo es capaz de detectar algunos ataques, genera muchos falsos positivos. Este resultado es consistente con lo observado en la matriz de confusión y la curva ROC del modelo SVM. A pesar de su sensibilidad, su utilidad práctica se ve limitada sin ajustes adicionales.

### Conclusión:

Los resultados confirman que, para este dataset, los modelos Random Forest y Regresión Logística ofrecen una detección muy precisa del tráfico malicioso. La regresión logística, además, lo consigue con una estructura más sencilla y un coste computacional más bajo, lo que la convierte en una opción especialmente interesante cuando se requiere interpretabilidad o rapidez. SVM, aunque potente teóricamente, no resulta competitivo en este caso sin un mayor ajuste y entrenamiento completo.


In [5]:
import time

# Función para calcular tiempo total y promedio por muestra
def medir_tiempo_inferencia(modelo, X_test):
    inicio = time.time()
    _ = modelo.predict(X_test)
    fin = time.time()
    tiempo_total = fin - inicio
    tiempo_promedio = tiempo_total / len(X_test)
    return round(tiempo_total, 4), round(tiempo_promedio, 6)

# Ejecutar mediciones
tiempos = []
for nombre, modelo in [("Random Forest", rf_model), ("Logistic Regression", lr_model), ("SVM (RBF, 10k)", svm_model)]:
    total, promedio = medir_tiempo_inferencia(modelo, X_test)
    tiempos.append((nombre, total, promedio))

# Crear tabla con resultados
df_tiempos = pd.DataFrame(tiempos, columns=["Modelo", "Tiempo total (s)", "Tiempo por muestra (s)"])
display(df_tiempos)

,Modelo,Tiempo total (s),Tiempo por muestra (s)
0,Random Forest,0.1627,0.000004
1,Logistic Regression,0.0064,0.000000
2,"SVM (RBF, 10k)",11.3765,0.000255


## Análisis de Tiempos de Inferencia

La siguiente tabla muestra el tiempo total requerido por cada modelo para realizar predicciones sobre 40,000 muestras de prueba, así como el tiempo promedio de inferencia por muestra:

| Modelo               | Tiempo total (s) | Tiempo por muestra (s) |
|----------------------|------------------|-------------------------|
| Random Forest        | 0.1627           | 0.000004                |
| Logistic Regression  | 0.0064           | 0.000000                |
| SVM (RBF, 10k)       | 11.3765          | 0.000255                |

### Interpretación:

- **Regresión Logística** es el modelo más rápido, con un tiempo prácticamente instantáneo por muestra. Esto lo convierte en una opción excelente para entornos con restricciones de recursos o donde la latencia es crítica.

- **Random Forest** también presenta un rendimiento muy eficiente, con un tiempo de inferencia por muestra de solo 4 microsegundos. Dado su alto rendimiento en métricas de clasificación, este resultado lo posiciona como una opción muy competitiva para detección de intrusiones en tiempo real.

- **SVM (entrenado con 10k muestras)** es el más lento, tardando más de 11 segundos en procesar las 40,000 muestras del conjunto de prueba. Su tiempo de inferencia por muestra es 50 veces superior al de Random Forest. Esto limita su aplicabilidad en entornos donde se requiera respuesta inmediata o se procesen flujos de datos continuos.

### Conclusión:

Este análisis demuestra que, además de precisión y AUC, la eficiencia computacional es un factor clave para elegir el modelo más adecuado. En este caso, Regresión Logística y Random Forest ofrecen una combinación óptima de velocidad y rendimiento, mientras que SVM, aunque útil para detección agresiva, puede no ser viable en tiempo real sin optimización adicional.


In [6]:
import platform
import psutil
import pandas as pd

# Recoger especificaciones del sistema
specs = {
    "Sistema Operativo": platform.system(),
    "Versión OS": platform.version(),
    "Procesador": platform.processor(),
    "Núcleos físicos": psutil.cpu_count(logical=False),
    "Núcleos lógicos": psutil.cpu_count(logical=True),
    "RAM total (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2)
}

# Mostrar como tabla
specs_df = pd.DataFrame(list(specs.items()), columns=["Recurso", "Valor"])
display(specs_df)


,Recurso,Valor
0,Sistema Operativo,Linux
1,Versión OS,#53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan...
2,Procesador,x86_64
3,Núcleos físicos,4
4,Núcleos lógicos,8
5,RAM total (GB),15.42


## Especificaciones del Sistema de Evaluación

Las pruebas de inferencia se realizaron en el siguiente entorno:

- Sistema operativo y versión
- Procesador y cantidad de núcleos
- Memoria RAM total disponible

Esta información permite entender el contexto en el que se evaluó la eficiencia de los modelos, y sirve como referencia para reproducibilidad de los tiempos.

Los resultados pueden variar si se ejecutan en sistemas con características técnicas distintas.
